In [2]:
from obspy import read, Stream
import numpy as np
import random
import os

def time_shift(trace, max_shift):
    shift_amount = random.uniform(-max_shift, max_shift)
    return trace.copy().trim(starttime=trace.stats.starttime + shift_amount,
                             endtime=trace.stats.endtime + shift_amount)

def amplitude_scaling(trace, min_scale, max_scale):
    scale_factor = random.uniform(min_scale, max_scale)
    scaled_trace = trace.copy()
    scaled_trace.data = scaled_trace.data.astype(float) * scale_factor
    return scaled_trace

def augment_data(stream, n_augmentations, max_shift_range, min_scale_range, max_scale_range):
    augmented_streams = []
    for i in range(n_augmentations):
        augmented_traces = []
        for trace in stream:
            max_shift = random.uniform(*max_shift_range)
            min_scale = random.uniform(*min_scale_range)
            max_scale = random.uniform(*max_scale_range)
            augmented_trace = time_shift(trace, max_shift)
            augmented_trace = amplitude_scaling(augmented_trace, min_scale, max_scale)
            augmented_traces.append(augmented_trace)
        augmented_streams.append(augmented_traces)
    return augmented_streams


In [4]:
def calculate_parameter_ranges(stream):
    # Calculate max shift range based on the duration of the waveform
    total_duration = stream[-1].stats.endtime - stream[0].stats.starttime
    max_shift_range = (0.1 * total_duration, 0.2 * total_duration)

    # Calculate min and max scale ranges based on the maximum and minimum amplitudes in the waveform
    max_amplitude = max(np.abs(trace.data).max() for trace in stream)
    min_scale_range = (0.8, 0.9)
    max_scale_range = (1.1, 1.2)  # You can adjust these based on your specific needs

    return max_shift_range, min_scale_range, max_scale_range

# Directory containing original MiniSEED files
input_dir = "D:\Skripsi\SKRIPSI-FINALIZED-CODE-SIDANG\SKRIPSI-FINALIZED-CODE-SIDANG\Dataset\Dataset Model\Testing\GENDOL"


# Output directory for augmented MiniSEED files
output_dir = "D:\Skripsi\SKRIPSI-FINALIZED-CODE-SIDANG\SKRIPSI-FINALIZED-CODE-SIDANG\Dataset\Dataset Model\Testing\GENDOL"

# Iterate over MiniSEED files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".msd"):
        # Read original MiniSEED file
        stream = read(os.path.join(input_dir, filename))

        # Calculate parameter ranges based on the characteristics of the seismic data
        max_shift_range, min_scale_range, max_scale_range = calculate_parameter_ranges(stream)

        # Define augmentation parameters
        n_augmentations = 2  # Number of augmented datasets

        # Augment data and save to multiple MiniSEED files
        for i, augmented_traces in enumerate(augment_data(stream, n_augmentations, max_shift_range, min_scale_range, max_scale_range)):
            augmented_stream = Stream(traces=augmented_traces)
            output_filename = f"{os.path.splitext(filename)[0]}{i}.msd"
            output_path = os.path.join(output_dir, output_filename)
            augmented_stream.write(output_path, format="MSEED")